In [1]:
import cv2
import numpy as np
import imageio

In [2]:
# Function to get corner points of an image
def getCornerPoints(img):
    w, h = img.shape[:2]
    cornerPoints = [(0, 0), (w, 0), (0, h), (w, h)]
    return cornerPoints
    
#Function to solve linear equation in two variables
def solveLinEq(equation):
    a1, b1, c1 = equation[0]
    a2, b2, c2 = equation[1]
    # Equations are:-
    # a1*x + b1*y = c1 ----- (1)
    # a2*x + b2*y = c2 ----- (2)
    x = (b1*c2 - b2*c1)/(b1*a2 - b2*a1) # b1*(2) - b2*(1)
    y = (c1*a2 - c2*a1)/(b1*a2 - b2*a1) # a2*(1) - a1*(2)
    return (x, y)

#Function to get the centre and radius of the circumcircle of a triangle
def circumcircle(triangle):
    x1, y1 = triangle[0]
    x2, y2 = triangle[1]
    x3, y3 = triangle[2]
    # Circumcenter of a triangle is where the perpendicular bisectors of any two sides intersect
    equation = np.array([[2*(x1 - x2), 2*(y1 - y2), x1**2 + y1**2 - x2**2 - y2**2], [2*(x1 - x3), 2*(y1 - y3), x1**2 + y1**2 - x3**2 - y3**2]], dtype = np.float64)
    center = solveLinEq(equation)
    radius = np.sqrt((center[0] - x1)**2 + (center[1] - y1)**2)
    return [center, radius]

#Function to check if a point lies in a circle
def insideCircle(center, radius, point):
    h, k = center
    x, y = point
    if (x - h)**2 + (y - k)**2 - radius**2 <= 0:
        return True
    else:
        return False

#Function to find area of the triangle
def area_triangle(x1,y1,x2,y2,x3,y3):
    return abs((x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2))/2)
    
#Function to check if a point lies inside a triangle
def insideTriangle(triangle_coord, x, y):
    x1=triangle_coord[0][0]                         #If the point lies inside the triangle then,
    y1=triangle_coord[0][1]                         #....the sum of the areas of the triangles
    x2=triangle_coord[1][0]                         #....formed between the edges of the current
    y2=triangle_coord[1][1]                         #....triangle and the point equals the area 
    x3=triangle_coord[2][0]                         #....of the current triangle
    y3=triangle_coord[2][1]
    A = area_triangle(x1, y1, x2, y2, x3, y3) 
    A1 = area_triangle(x, y, x2, y2, x3, y3) 
    A2 = area_triangle(x1, y1, x, y, x3, y3)  
    A3 = area_triangle(x1, y1, x2, y2, x, y) 
    if(A == (A1 + A2 + A3)): 
        return True
    else: 
        return False
    
def reverse(tup):
    new_tup = tup[::-1]
    return new_tup
    
def delaunay(points):
    # Dictionary for triangles where key is tuple of vertices and value is tuple (circumcenter, circumradius)
    triangles = {}

    superTriangle = ((0, 10000), (10000, -10000), (-10000, -10000))
    triangles[superTriangle] = circumcircle(superTriangle)
    
    for point in points:
        edges = []
        wrong_triangles = []
        # Checking triangles which violate the Delaunay Triangulation condition upon addition of point
        for t in triangles:
            center = triangles[t][0]
            radius = triangles[t][1]
            if(insideCircle(center, radius, point)):
                # Adding the wrong triangles in the list
                wrong_triangles.append(t)
                # Calculating the edges formed by the wrong triangles
                t_edge = [(t[0], t[1]), (t[1], t[2]), (t[2], t[0])]
                for e in t_edge:
                    if e in edges:
                        edges.remove(e)
                        continue
                    e1 = reverse(e)
                    if e1 in edges:
                        edges.remove(e1)
                        continue
                    edges.append(e)
        # Remove the wrong triangles from the triangles dictionary
        for t in wrong_triangles:
            del triangles[t]
        # Adding the new triangles formed by the edges and the new point
        for e in edges:
            t = (point, e[0], e[1])
            triangles[t] = circumcircle(t)
            
    final_triangles = []
    # Remove the triangles formed by the vertices of the super triangle
    for t in triangles:
        if superTriangle[0] in t or superTriangle[1] in t or superTriangle[2] in t:
            continue
        else:
            final_triangles.append(t)
            
    return final_triangles

# Function to draw triangles over the given image
def drawTriangles(img, triangles, color):
    for t in triangles:
        cv2.line(img, reverse(t[0]), reverse(t[1]), color, 2)
        cv2.line(img, reverse(t[1]), reverse(t[2]), color, 2)
        cv2.line(img, reverse(t[2]), reverse(t[0]), color, 2)

In [3]:
# Taking the images 
img1 = cv2.imread("input/source.jpg"); img2 = cv2.imread("input/target.jpg")
img1_copy = img1.copy(); img2_copy = img2.copy()

# Tie-points for both the images
tiePoints1 = getCornerPoints(img1); tiePoints2 = getCornerPoints(img2)

In [4]:
x = 0
if x == 0:
    # Taking tie-points as user input through mouse clicks
    def mouse_click_img1(event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(img1_copy, (x,y), 3, (255,0,0), -1)
        if event == cv2.EVENT_LBUTTONUP:
            tiePoints1.append((y,x))
        cv2.imshow("ImageA", img1_copy)

    def mouse_click_img2(event,x,y,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(img2_copy, (x,y), 3, (0,0,255), -1)
        if event == cv2.EVENT_LBUTTONUP:
            tiePoints2.append((y,x))
        cv2.imshow("ImageB", img2_copy)

    cv2.imshow("ImageA", img1_copy)
    cv2.imshow("ImageB", img2_copy)
    cv2.moveWindow("ImageA", 0, 0)
    cv2.moveWindow("ImageB", 700, 0)
    cv2.setMouseCallback("ImageA", mouse_click_img1)
    cv2.setMouseCallback("ImageB", mouse_click_img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
if x == 1:
    # Taking tie-points as user input through a text file
    with open('tiePoints.txt') as file:
        for line in file:
            xA, yA, xB, yB = line.split()
            tiePoints1.append((xA, yA)); tiePoints2.append((xB, yB))

    for point in tiePoints1:
        cv2.circle(img1_copy, point, 2, (255, 0, 0), cv2.FILLED, cv2.LINE_AA, 0)
    for point in tiePoints2:
        cv2.circle(img2_copy, point, 2, (0, 0, 255), cv2.FILLED, cv2.LINE_AA, 0)
    cv2.imshow("ImageA", img1_copy)
    cv2.imshow("ImageB", img2_copy)
    cv2.moveWindow("ImageA", 100, 0)
    cv2.moveWindow("ImageB", 700, 0)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
# TODO: Automatically detecting appropriate tie-points for both the images
if x == 2:
    pass

In [7]:
# Saving the images depicting control points given by user
cv2.imwrite("output/ImageA_tp.jpg", img1_copy)
cv2.imwrite("output/ImageB_tp.jpg", img2_copy)

True

In [8]:
# Triangulating the tie-points using Delaunay Triangulation
triangle_img1 = delaunay(tiePoints1)
triangle_img2 = []
# Finding the triangles in the final image corresponding those of the inital image
for t in triangle_img1:
    e0 = tiePoints2[tiePoints1.index(t[0])]
    e1 = tiePoints2[tiePoints1.index(t[1])]
    e2 = tiePoints2[tiePoints1.index(t[2])]
    triangle_img2.append((e0,e1,e2))

# Printing the Triangles over the Image
drawTriangles(img1_copy, triangle_img1, (255, 0, 0))
drawTriangles(img2_copy, triangle_img2, (0, 0, 255))

cv2.imshow("ImageA", img1_copy)
cv2.imshow("ImageB", img2_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Saving the triangulated images
cv2.imwrite("output/ImageA_dt.jpg", img1_copy)
cv2.imwrite("output/ImageB_dt.jpg", img2_copy)

True

In [9]:
#Number of frames
no_of_frames = 100
#Array of all the frames
frames = np.array([np.zeros_like(img1)]*no_of_frames)
#Setting the inital and final frame to Initial and Final Image respectively
frames[0] = img1
frames[no_of_frames-1] = img2

In [10]:
#List to store triangle vertices of the frames
frames_triangle_coord = []
#List to store affine basis of each frame of each triangle
frames_affine_basis = []

for k in range(len(frames)):
    #List to store triangle coordinates of the current frame
    frame_triangle_coord = []
    #List to store affine basis of the triangles of current frame
    frame_affine_basis = []
    for trngl,trngln in zip(triangle_img1,triangle_img2):
            p0_x = trngl[0][0]
            p0_y = trngl[0][1]
            p1_x = trngl[1][0]
            p1_y = trngl[1][1]
            p2_x = trngl[2][0]
            p2_y = trngl[2][1]
            
            p0_xn = trngln[0][0]
            p0_yn = trngln[0][1]
            p1_xn = trngln[1][0]
            p1_yn = trngln[1][1]
            p2_xn = trngln[2][0]
            p2_yn = trngln[2][1]
            
            #Linearly Interpolating the control points to find the corresponding control points in intermediate frames
            p0_xk = int(((no_of_frames-k)/no_of_frames)*p0_x + (k/no_of_frames)*p0_xn)
            p0_yk = int(((no_of_frames-k)/no_of_frames)*p0_y + (k/no_of_frames)*p0_yn)
            p1_xk = int(((no_of_frames-k)/no_of_frames)*p1_x + (k/no_of_frames)*p1_xn)
            p1_yk = int(((no_of_frames-k)/no_of_frames)*p1_y + (k/no_of_frames)*p1_yn)
            p2_xk = int(((no_of_frames-k)/no_of_frames)*p2_x + (k/no_of_frames)*p2_xn)
            p2_yk = int(((no_of_frames-k)/no_of_frames)*p2_y + (k/no_of_frames)*p2_yn)
            
            #Finding the affine basis of the current triangle
            e1_xk = p1_xk - p0_xk
            e1_yk = p1_yk - p0_yk
            e2_xk = p2_xk - p0_xk
            e2_yk = p2_yk - p0_yk
            
            
            frame_affine_basis.append(((e1_xk,e1_yk),(e2_xk,e2_yk)))
            frame_triangle_coord.append(((p0_xk,p0_yk),(p1_xk,p1_yk),(p2_xk,p2_yk)))
    frames_triangle_coord.append(frame_triangle_coord)
    frames_affine_basis.append(frame_affine_basis)

In [11]:
#Traversing over each frame
for k in range(1,len(frames)-1):
    #Traversing over each pixel of the frame
    for x in range(frames[k].shape[0]):
        for y in range(frames[k].shape[1]):
            count=0
            #Finding which triangle the pixel belongs to in the current pixel
            for triangle_coord,affine_basis in zip(frames_triangle_coord[k-1],frames_affine_basis[k-1]):
                if(insideTriangle(triangle_coord,x,y)):
                    #Calculating alpha and beta i.e. affine coordinates
                    equation = np.array([[affine_basis[0][0],affine_basis[1][0],x-triangle_coord[0][0]],[affine_basis[0][1],affine_basis[1][1],y-triangle_coord[0][1]]])
                    z = solveLinEq(equation)
                    alpha = z[0]
                    beta = z[1]
                    
                    #Calculating corresponding points in source and destination image
                    p_x = alpha*frames_affine_basis[0][count][0][0] + beta*frames_affine_basis[0][count][1][0] + frames_triangle_coord[0][count][0][0]
                    p_y = alpha*frames_affine_basis[0][count][0][1] + beta*frames_affine_basis[0][count][1][1] + frames_triangle_coord[0][count][0][1]
                    p_xn = alpha*frames_affine_basis[no_of_frames-1][count][0][0] + beta*frames_affine_basis[no_of_frames-1][count][1][0] + frames_triangle_coord[no_of_frames-1][count][0][0]
                    p_yn = alpha*frames_affine_basis[no_of_frames-1][count][0][1] + beta*frames_affine_basis[no_of_frames-1][count][1][1] + frames_triangle_coord[no_of_frames-1][count][0][1]
                    
                    #Adjusting the values
                    if(p_x>=frames[k].shape[0]):
                        p_x = frames[k].shape[0]-1
                    if(p_y>=frames[k].shape[1]):
                        p_y = frames[k].shape[1]-1
                    if(p_xn>=frames[k].shape[0]):
                        p_xn = frames[k].shape[0]-1
                    if(p_yn>=frames[k].shape[1]):
                        p_yn = frames[k].shape[1]-1
                        
                    #Assigning the pixel intensity
                    pixel_colour = ((1-k/no_of_frames)*frames[0][int(p_x),int(p_y)] + (k/no_of_frames)*frames[no_of_frames-1][int(p_xn),int(p_yn)]).astype(int)
                    frames[k][x][y] = pixel_colour
                    break
                else:
                    count+=1

In [ ]:
for i in range(no_of_frames):
    frames[i] = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB)

imageio.mimsave('output/movie.gif', frames, fps = 50)